In [4]:
import pandas as pd
import numpy as np
import math
from google.colab import files
import pprint

uploaded = files.upload()
file_name = list(uploaded.keys())[0]
data = pd.read_csv(file_name)
data.columns = data.columns.str.strip()

print(data.head())
target = "Play Tennis"

def ent(col):
    vals, counts = np.unique(col, return_counts=True)
    return sum([-counts[i] / sum(counts) * math.log2(counts[i] / sum(counts)) for i in range(len(vals))])

def info_gain(data, attr, target):
    total_ent = ent(data[target])
    vals, counts = np.unique(data[attr], return_counts=True)
    weighted_ent = sum([(counts[i] / sum(counts)) * ent(data[data[attr] == vals[i]][target]) for i in range(len(vals))])
    return total_ent - weighted_ent

def ID3(data, orig_data, attrs, target, parent_class=None):
    if len(np.unique(data[target])) <= 1:
        return np.unique(data[target])[0]
    elif len(data) == 0:
        return np.unique(orig_data[target])[np.argmax(np.unique(orig_data[target], return_counts=True)[1])]
    elif len(attrs) == 0:
        return parent_class
    else:
        parent_class = np.unique(data[target])[np.argmax(np.unique(data[target], return_counts=True)[1])]
        gains = [info_gain(data, attr, target) for attr in attrs]
        best_attr = attrs[np.argmax(gains)]
        tree = {best_attr: {}}
        attrs.remove(best_attr)

        for val in np.unique(data[best_attr]):
            sub_data = data[data[best_attr] == val]
            subtree = ID3(sub_data, orig_data, attrs, target, parent_class)
            tree[best_attr][val] = subtree

        return tree

attrs = list(data.columns[:-1])
dec_tree = ID3(data, data, attrs, target)

print("\nDecision Tree Structure:")
pprint.pprint(dec_tree)


Saving PlayTennis.csv to PlayTennis (3).csv
    Outlook Temperature Humidity    Wind Play Tennis
0     Sunny         Hot     High    Weak          No
1     Sunny         Hot     High  Strong          No
2  Overcast         Hot     High    Weak         Yes
3      Rain        Mild     High    Weak         Yes
4      Rain        Cool   Normal    Weak         Yes

Decision Tree Structure:
{'Outlook': {'Overcast': 'Yes',
             'Rain': {'Wind': {'Strong': 'No', 'Weak': 'Yes'}},
             'Sunny': {'Humidity': {'High': 'No', 'Normal': 'Yes'}}}}
